# M2 - 220 - Experiment #2 - Joint-Labelling with noisy dataset

In [ ]:
#!pip install --upgrade transformers datasets spacy transformers[sentencepiece] seqeval

## Initialisation
Set the BASE path.
If run on Google Colab, will also mout Google Drive to the moutpoint given below

In [ ]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset_ICDAR"
  OUT_BASE = BASE / "res_ICDAR/method_2"
else:
  BASE = Path().resolve() # Directory of this approach
  #Adapt this to your situation
  DATASETS = Path('../dataset_ICDAR').resolve() #Where your data are located befor Dataset object creation
  OUT_BASE = Path('../res_ICDAR/method_2').resolve() #Where you save the results of this notebook

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

## Constants

**Don't forget to check tokenizer name in *model_util_IO.py* and *model_util_IOB2.py* files (same name as model) !**

In [ ]:
RUN_CAMEMBERT_IO = False
RUN_CAMEMBERT_IOB2 = False
#Can't run together because of convert_tokenizer_
RUN_PTRN_CAMEMBERT_IO = True
RUN_PTRN_CAMEMBERT_IOB2 = True

# Number of times a model will be trained & evaluated on each a dataset
N_RUNS = 5

## Parameters

In [ ]:
# COMMON CONSTANTS

TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100,
    "max_steps": 5000,
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 1e-5,
    "load_best_model_at_end": True,
    "greater_is_better":True,
    "metric_for_best_model": "f1",
    "save_strategy": "steps",
    "save_steps": 100,
    "save_total_limit": 1
}

In [ ]:
import pandas as pd
from datasets import load_from_disk

#Print examples from datasets
def loadExample(INPUT_DIR,set_length:int,i:int,subset:str):
    set_ = load_from_disk(INPUT_DIR / f"huggingface_{set_length}")
    data = {"tokens": set_[subset][i]["tokens"],
            "labels": set_[subset][i]["ner_tags"]}
    df = pd.DataFrame.from_dict(data)
    print(df)

# 221 - Train & eval : IO Pero OCR dataset with CamemBERT model

In [ ]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "io"
FOLDER = "221-camembert-ner-joint-labelling-io"

### 221.1 Load IO dataset

In [ ]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

### Example

In [ ]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

### 221.2 Fine-tuning with IO labels - train & eval

In [ ]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IO import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [ ]:
import time
import datetime

if RUN_CAMEMBERT_IO:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

# 222 - Train & eval : IOB2 Pero OCR dataset with CamemBERT model

In [ ]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "iob2"
FOLDER = "222-camembert-ner-joint-labelling-iob2"

### 222.1 Load IOB2 dataset

In [ ]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

### Example

In [ ]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

### 222.2 Fine-tuning with IOB2 labels - train & eval

In [ ]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")
            
            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [ ]:
import time
import datetime

if RUN_CAMEMBERT_IOB2:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IOB2 labels")

# 223 - Train & eval : IO Pero OCR dataset with Pretrained CamemBERT model

In [ ]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "io"
FOLDER = "223-pretrained-camembert-ner-joint-labelling-io"

### 223.1 Load IO dataset

In [ ]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

In [ ]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

### 223.2 Fine-tuning with IO labels - train & eval

In [ ]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IO import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [ ]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IO:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH


    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

# 224 - Train & eval : IOB2 Pero dataset with Pretrained CamemBERT model

In [ ]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "iob2"
FOLDER = "224-pretrained-camembert-ner-joint-labelling-iob2"

### 224.1 Load IOB2 dataset

In [ ]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

In [ ]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

### 224.2 Fine-tuning with IOB2 labels - train & eval

In [ ]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer

torch.cuda.empty_cache()

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [ ]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IOB2:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IOB2 labels")